In [18]:
import numpy as np
import os

In [19]:
def write_pbs_head(fpbs, job_name):
    fpbs.write('#! /bin/bash\n')
    fpbs.write('#PBS -M zhangji@csrc.ac.cn\n')
    fpbs.write('#PBS -l nodes=1:ppn=24\n')
    fpbs.write('#PBS -l walltime=72:00:00\n')
    fpbs.write('#PBS -q common\n')
    fpbs.write('#PBS -N %s\n' % job_name)
    fpbs.write('\n')
    fpbs.write('cd $PBS_O_WORKDIR\n')
    fpbs.write('\n')
    

In [23]:
# case 1, traverse all parameters, inf space
maxtheta_list = [10, 50, 100, 500, 1000, 5000, 10000, 50000]
nphi_list = [100, 500, 1000, 5000, 10000, 20000]
nthfct_list = [1, 10, 100]  # nth = nthfct * maxtheta
job_dir = 'infhlx_20180510'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)

t_name = os.path.join(job_dir, 'run.sh')
with open(t_name, 'w') as frun:
    # create .pbs file
    for maxtheta in maxtheta_list:
        for nphi in nphi_list:
            for nthfct in nthfct_list:
                nth = nthfct * maxtheta
                job_name = 'infhlx%06d_%06d_%04d' % (maxtheta, nphi, nthfct)
                t_name = os.path.join(job_dir, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)
                    fpbs.write('mpirun -n 24 python ')
                    fpbs.write(' ../try_infhelix.py ')
                    fpbs.write(' -infhelix_maxtheta %f ' % maxtheta)
                    fpbs.write(' -infhelix_nnode %f ' % nphi)
                    fpbs.write(' -infhelix_ntheta %f ' % nth)
                    fpbs.write(' > %s.txt' % job_name)
                
    # write to .sh file
                    frun.write('qsub %s.pbs\n' % job_name)
    
    frun.write('\n')

In [37]:
# case 2, maxtheta varing, inf space
maxtheta_list = 10 ** np.arange(1, 8)
nphi_list = [100, ]
nthfct_list = [30, ]  # nth = nthfct * maxtheta
job_dir = 'infhlx_20180511'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)

t_name = os.path.join(job_dir, 'run.sh')
with open(t_name, 'w') as frun:
    # create .pbs file
    for maxtheta in maxtheta_list:
        for nphi in nphi_list:
            for nthfct in nthfct_list:
                nth = nthfct * maxtheta
                job_name = 'infhlx_maxtheta1E%02d' % np.log10(maxtheta)
                t_name = os.path.join(job_dir, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)
                    fpbs.write('mpirun -n 24 python ')
                    fpbs.write(' ../try_infhelix.py ')
                    fpbs.write(' -infhelix_maxtheta %f ' % maxtheta)
                    fpbs.write(' -infhelix_nnode %f ' % nphi)
                    fpbs.write(' -infhelix_ntheta %f ' % nth)
                    fpbs.write(' -ksp_max_it 1000')
                    fpbs.write(' > %s.txt' % job_name)
                
    # write to .sh file
                    frun.write('qsub %s.pbs\n' % job_name)
    
    frun.write('\n')


In [36]:
# case 2, maxtheta varing, inf space
maxtheta_list = 10 ** np.arange(1, 8)
nphi_list = [100, ]
nthfct_list = [30, ]  # nth = nthfct * maxtheta
job_dir = 'helixInPipe_20180512'
if not os.path.exists(job_dir):
    os.makedirs(job_dir)

t_name = os.path.join(job_dir, 'run.sh')
with open(t_name, 'w') as frun:
    # create .pbs file
    for maxtheta in maxtheta_list:
        for nphi in nphi_list:
            for nthfct in nthfct_list:
                nth = nthfct * maxtheta
                job_name = 'helixInPipe_1E%1d' % np.log10(maxtheta)
                t_name = os.path.join(job_dir, '%s.pbs' % job_name)
                with open(t_name, 'w') as fpbs:
                    write_pbs_head(fpbs, job_name)
                    fpbs.write('mpirun -n 24 python ')
                    fpbs.write(' ../helixInPipe.py ')
                    fpbs.write(' -infhelix_maxtheta %f ' % maxtheta)
                    fpbs.write(' -infhelix_nnode %f ' % nphi)
                    fpbs.write(' -infhelix_ntheta %f ' % nth)
                    fpbs.write(' -ksp_max_it 1000')
                    fpbs.write(' > %s.txt' % job_name)
                
    # write to .sh file
                    frun.write('qsub %s.pbs\n' % job_name)
    
    frun.write('\n')
